# Launch post-run analysis jobs
Code to create a batch script for launching jobs on cori
Sep 1, 2020

In [1]:
import os
import glob
import time
import subprocess as sp
import numpy as np


In [2]:
curr_dir=os.getcwd()
print(curr_dir)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/5_3d_cgan


In [3]:
val_files={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/',
          '512':''
}

In [4]:
results_loc='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/'

In [5]:
ls /global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/

20210223_210217_3dcgan_predict_0.8_m2/
20210227_050213_3dcgan_predict_0.65_m2/
20210309_200006_3dcgan_predict_0.65_m2/
20210313_050345_3dcgan_predict_0.65_m2/


In [6]:
# glob.glob(results_loc+'*')

In [6]:
ip_folder='20210313_050345_3dcgan_predict_0.65_m2'

In [7]:
### Default dictionary
img_size=64
dict_pars={'job_name':'analysis_3dcgan', 'queue':'debug','bins':'uneven','cores':'64',
           'time':'00:30:00'}

dict_pars['val_file']=val_files[str(img_size)]
dict_pars['ip_folder']=results_loc+ip_folder

print(dict_pars)

{'job_name': 'analysis_3dcgan', 'queue': 'debug', 'bins': 'uneven', 'cores': '64', 'time': '00:30:00', 'val_file': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/', 'ip_folder': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210313_050345_3dcgan_predict_0.65_m2'}


In [8]:
assert os.path.isdir(dict_pars['ip_folder'])

In [9]:
bash_strg='''#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos={queue}
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time={time}
#SBATCH --job-name={job_name}

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

conda activate v3
code_dir='/global/homes/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/5_3d_cgan/1_main_code'

python $code_dir/post_analysis_pandas.py -f {ip_folder}  -v {val_file} -c {cores} -bin {bins}
conda deactivate
echo "--end date" `date` `date +%s`
'''.format(**dict_pars)

In [10]:
# bash_strg

In [11]:
### Write bash file
staging_loc='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/'
fname='batch_analysis_3dcgan.sh'
filename=staging_loc+fname
print(filename)
with open (filename,'w') as f:
    f.write(bash_strg)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/batch_analysis_3dcgan.sh


In [12]:
### Move to staging locations in project space:
os.chdir(staging_loc)

In [13]:
%%bash -s "$filename" ## Use python variable in bash
cat $1
chmod +x $1

#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos=debug
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time=00:30:00
#SBATCH --job-name=analysis_3dcgan

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

conda activate v3
code_dir='/global/homes/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/5_3d_cgan/1_main_code'

python $code_dir/post_analysis_pandas.py -f /global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210313_050345_3dcgan_predict_0.65_m2  -v /global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/ -c 64 -bin uneven
conda deactivate
echo "--end date" `date` `date +%s`


### Submit jobs to cori queue

In [14]:
%%bash -s "$filename" ## Use python variable in bash
sbatch $1

Submitted batch job 40506640
